In [229]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [230]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [231]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [232]:
# get only the column "Survived" 
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

% of women who survived: 0.7420382165605095


In [233]:
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

% of men who survived: 0.18890814558058924


In [234]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

In [235]:
# verifier léquilibre des data dentrainement

# variable cible (à prédire)
y = train_data["Survived"]
print(y.value_counts(normalize=True))

# conclusion: déséquilibré

Survived
0    0.616162
1    0.383838
Name: proportion, dtype: float64


In [236]:
# Check for NaN values in the entire dataset
print(train_data.isna().sum())
missing_percentage = train_data.isna().sum() / len(train_data) * 100
print(missing_percentage)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            19.865320
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
Cabin          77.104377
Embarked        0.224467
dtype: float64


In [237]:
train_data['Age'] = train_data.groupby(['Pclass', 'Sex'])['Age'].transform(lambda x: x.fillna(x.median()))

In [238]:
# Essaie pour améliorer les résultats
# Créer des groupes d'âge
bins = [0, 12, 18, 35, 60, 100]  # Définir les intervalles d'âge
labels = ['Enfant', 'Adolescent', 'Jeune Adulte', 'Adulte', 'Senior']  # Noms des groupes
train_data['AgeGroup'] = pd.cut(train_data['Age'], bins=bins, labels=labels)

# Vérifier les résultats
print(train_data[['Age', 'AgeGroup']].head())

# Encodage one-hot de la colonne AgeGroup
train_data = pd.get_dummies(train_data, columns=['AgeGroup'], drop_first=True)


    Age      AgeGroup
0  22.0  Jeune Adulte
1  38.0        Adulte
2  26.0  Jeune Adulte
3  35.0  Jeune Adulte
4  35.0  Jeune Adulte


In [239]:
# Créer la colonne FamilySize
train_data['FamilySize'] = train_data['SibSp'] + train_data['Parch'] + 1

# Créer une colonne indiquant si la personne est seule
train_data['IsAlone'] = (train_data['FamilySize'] == 1).astype(int)

In [240]:
# Données de test
# Grouper l'âge dans les données de test
test_data['AgeGroup'] = pd.cut(test_data['Age'], bins=bins, labels=labels)

# Encodage one-hot pour AgeGroup
test_data = pd.get_dummies(test_data, columns=['AgeGroup'], drop_first=True)

# Créer les colonnes FamilySize et IsAlone
test_data['FamilySize'] = test_data['SibSp'] + test_data['Parch'] + 1
test_data['IsAlone'] = (test_data['FamilySize'] == 1).astype(int)

In [241]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,AgeGroup_Adolescent,AgeGroup_Jeune Adulte,AgeGroup_Adulte,AgeGroup_Senior,FamilySize,IsAlone
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,False,True,False,False,2,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,False,False,True,False,2,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,False,True,False,False,1,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,False,True,False,False,2,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,False,True,False,False,1,1


In [242]:
# features we want to look at
age_group_columns = [col for col in train_data.columns if 'AgeGroup_' in col]

features = ["Pclass", "Sex", "FamilySize"] + age_group_columns
#features = ["Pclass", "Sex", "FamilySize"]

# get_dummies: convertit les variables catégoriques (comme Sex) en variables numériques via un encodage One-Hot Encoding
# sex -> sex-Female, sex_Male
# randomForestClassifier only uses numeric data
X = pd.get_dummies(train_data[features])


In [243]:
print(features)

['Pclass', 'Sex', 'FamilySize', 'AgeGroup_Adolescent', 'AgeGroup_Jeune Adulte', 'AgeGroup_Adulte', 'AgeGroup_Senior']


In [244]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1)

X_test = pd.get_dummies(test_data[features])

In [245]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=1)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

In [246]:
model = RandomForestClassifier(n_estimators=300, max_depth=5, random_state=1)
#model = XGBClassifier(n_estimators=300, max_depth=5, random_state=1)

# solution : class_weight= "balanced" va affecter des poids
# autre solution : Sur-échantillonnage de la classe minoritaire (exemple : SMOTE) ou Sous-échantillonnage de la classe majoritaire.

In [247]:
model.fit(X_train_balanced, y_train_balanced)

RandomForestClassifier(max_depth=5, n_estimators=300, random_state=1)

In [248]:
# évaluer les performance du model sur les données d'entrainement
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Prédictions sur l'ensemble de validation
y_val_pred = model.predict(X_val)

# Calcul de la précision
accuracy = accuracy_score(y_val, y_val_pred)
print("Précision sur l'ensemble de validation :", accuracy)

# Calcul du F1-score
f1 = f1_score(y_val, y_val_pred)
print("F1-score :", f1)

# Matrice de confusion
conf_matrix = confusion_matrix(y_val, y_val_pred)
print("Matrice de confusion :\n", conf_matrix)


Précision sur l'ensemble de validation : 0.8156424581005587
F1-score : 0.7659574468085105
Matrice de confusion :
 [[92 14]
 [19 54]]


In [249]:
# optionnel
# validation croisé pour choisir le max_depth de la random forest
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [5, 10, 15, None],
    'n_estimators': [100, 200, 300],
    'class_weight': [None, 'balanced']
}
grid_search = GridSearchCV(RandomForestClassifier(random_state=1), param_grid, cv=5)
grid_search.fit(X_train_balanced, y_train_balanced)

print("Meilleurs paramètres :", grid_search.best_params_)


Meilleurs paramètres : {'class_weight': None, 'max_depth': 15, 'n_estimators': 100}


In [250]:
# Une profondeur trop grande peut conduire à un surapprentissage (overfitting), où le modèle est trop spécifique aux données dentraînement.
# Une profondeur trop petite peut conduire à un sous-apprentissage (underfitting), où le modèle nest pas assez complexe pour capturer les relations dans les données.
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [251]:
# en cas de bias
feature_importances = model.feature_importances_
print(dict(zip(X.columns, feature_importances)))

{'Pclass': 0.20901478280276875, 'FamilySize': 0.14771825913547365, 'AgeGroup_Adolescent': 0.012794667192983281, 'AgeGroup_Jeune Adulte': 0.038003588610235124, 'AgeGroup_Adulte': 0.02169144776595443, 'AgeGroup_Senior': 0.010914924773534096, 'Sex_female': 0.2970671150488432, 'Sex_male': 0.2627952146702075}
